## Streaming and visualizing data from the cloud database 

In [76]:
import pandas as pd
from matplotlib import pyplot as plt
from influxdb import InfluxDBClient
from pandas import DataFrame, Series
from pandas.io.json import json_normalize
from influxdb import InfluxDBClient
from datetime import datetime, timedelta
import plotly.graph_objs as go
import numpy as np 
from dash.dependencies import Input, Output, State
import pytz
import tzlocal
from sklearn.cluster import KMeans
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.express as px

## Live data 

In [77]:
# streaming data from the cloud database
client = InfluxDBClient(host='34.105.67.126', port=8086, username='admin', password='BSSty1KTfj9MSSJy', database='muringato_arduino',ssl=True)
query = "select * from \"muringato_arduino\" where \"sensor\" =\'arduino2\'"
result = client.query(query)
result_list = list(result.get_points())
df = pd.DataFrame(result_list)
df[['time']] = df[['time']].apply(pd.to_datetime)
#df

c:\users\folio\anaconda3\envs\dsail2\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



## Convert to your own timezone 

In [78]:
df[['time']] = df[['time']].apply(pd.to_datetime)
df1 = df.set_index('time')
df2 = df1['2021-07-27':'2021-07-27']
df_zone = df2.tz_convert('Africa/Nairobi')
df_plot = df_zone.reset_index()
#df

## Plotting the Level data
- The level data from the dataframe 

In [79]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot['time'], y=df_plot['level'],
                    mode='lines+markers',
                    name='lines+markers'))
fig.layout=go.Layout(
    plot_bgcolor = '#000000', 
    paper_bgcolor = '#000000',
    font=dict(family = "Courier New, monospace",size = 13, color = "#D2691E"),
    title = 'Water-level', 
    yaxis = {'autorange': False, 'range': [0, 200] ,'ticksuffix': ' CM' ,'gridcolor':'#FFFFFF','title': {'text':'Water-level(Meters)'}},
    xaxis=go.layout.XAxis(gridcolor = '#FFFFFF',title_text = "TIME", rangeslider = dict( visible = True), type = "date")
)
fig.show()

## Anomaly detection using Kmeans. 

In [80]:
df_plot[['time']] = df_plot[['time']].apply(pd.to_datetime)
water_level = np.array(df_plot['level']).reshape(-1, 1)
num_of_clusters = 3
water_level_pred = KMeans(n_clusters = num_of_clusters, random_state = 123).fit_predict(water_level)
water_level_pred2 = list(water_level_pred)
majority_index = (max(set(water_level_pred2), key = water_level_pred2.count))
kmeans_index = water_level_pred2
df_plot['kmeans_index'] = kmeans_index
df_plota2 = df_plot.drop(df_plot[df_plot.kmeans_index != majority_index].index)
df_after_kmeans = df_plota2.drop(['airtime', 'coding_rate', 'data_rate', 'frequency', 'gateway', 'rssi', 'sensor', 'snr', 'kmeans_index'], axis = 1)
#df_after_kmeans

In [81]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_after_kmeans['time'], y=df_after_kmeans['level'],
                    mode='lines+markers',
                    name='lines+markers'))
fig.layout=go.Layout(
    plot_bgcolor = '#000000', 
    paper_bgcolor = '#000000',
    font=dict(family = "Courier New, monospace",size = 13, color = "#D2691E"),
    title = 'Water-level', 
    yaxis = {'autorange': False, 'range': [0, 200] ,'ticksuffix': ' CM' ,'gridcolor':'#FFFFFF','title': {'text':'Water-level(Meters)'}},
    xaxis=go.layout.XAxis(gridcolor = '#FFFFFF',title_text = "TIME", rangeslider = dict( visible = True), type = "date")
)
fig.show()